In [ ]:
pip install jq

In [8]:
from pymongo import MongoClient
import os
from dotenv import load_dotenv
load_dotenv()

True

In [6]:

client = MongoClient(os.getenv('MONGODB_URI'), tls=True,
    tlsAllowInvalidCertificates=True)
db = client.get_database(os.getenv('DB_NAME'))
c_name = os.getenv('COLLECTION_NAME')
collection = db[c_name]

In [11]:
collection.delete_many({})

DeleteResult({'n': 0, 'electionId': ObjectId('7fffffff0000000000000140'), 'opTime': {'ts': Timestamp(1718131736, 7), 't': 320}, 'ok': 1.0, '$clusterTime': {'clusterTime': Timestamp(1718131736, 7), 'signature': {'hash': b':\x198R\xc3\x1f\x11zLN.\xb9\nP\xc0r(\x89\x92\xa8', 'keyId': 7327031572081672193}}, 'operationTime': Timestamp(1718131736, 7)}, acknowledged=True)

In [13]:
from langchain_community.document_loaders import JSONLoader
from langchain_community.embeddings import OpenAIEmbeddings
def metadata_func(record: dict, metadata: dict) -> dict:
    def column_retriever(ls):
        # cname = []
        # dtype = []
        cdesc = {}
        encode_values = {}
        for i in range(len(ls)):
            cdesc[ls[i].get("Column_Description")] = ls[i].get("Column_Name")
            if ls[i].get("encoded_values",-10000)!=-10000:
              encode_values[ls[i].get("Column_Name")] = ls[i].get("encoded_values")

        return encode_values, cdesc
    encode_values, cdesc = column_retriever(record.get("Columns"))

    metadata["Table_Name"] = record.get("Table_Name")
    metadata["Table_Description"] = record.get("Table_Description")
    metadata["Encoded_Values"] = str(encode_values)
    metadata["Column_Description"] = str(cdesc)

    return metadata

loader = JSONLoader(
    file_path="/content/tableinfo_new.json",
    jq_schema=".[].Table_Info[]",
    content_key="Table_Name",
    metadata_func=metadata_func,
)
data = loader.load()

In [14]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

openai_api_key = os.getenv("OPENAI_API_KEY")

embedding_function = OpenAIEmbeddings(
    openai_api_key=openai_api_key, model="text-embedding-ada-002")

In [15]:
from langchain_mongodb import MongoDBAtlasVectorSearch
vector_search = MongoDBAtlasVectorSearch(collection=collection, embedding=embedding_function)

# Add documents to the vector index
vector_search.add_documents(
    documents=data,
    index_name=os.getenv("ATLAS_VECTOR_SEARCH_INDEX_NAME")
)

[ObjectId('66689c8ff515f46de5e7f7ca'),
 ObjectId('66689c8ff515f46de5e7f7cb'),
 ObjectId('66689c8ff515f46de5e7f7cc'),
 ObjectId('66689c8ff515f46de5e7f7cd'),
 ObjectId('66689c8ff515f46de5e7f7ce'),
 ObjectId('66689c8ff515f46de5e7f7cf')]